In [ ]:
# ercot_volatility_analysis.py (Enhanced with diagnostics and GARCH-X)

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from arch import arch_model
from matplotlib import cm
from scipy.stats import ttest_ind, mannwhitneyu
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch

# === Load dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
full_df = pd.read_csv(file_path, parse_dates=['datetime'])

# === Define zones and colormap ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']
colorscale = cm.get_cmap('Blues', 4)

zone_colors = {
    'LZ_HOUSTON': 'rgba(40, 80, 180, 1.0)',   # custom dark blue
    'LZ_NORTH': 'rgba(80, 120, 200, 1.0)',
    'LZ_SOUTH': 'rgba(120, 160, 220, 1.0)',
    'LZ_WEST': 'rgba(160, 200, 240, 1.0)'
}

bess_start_date = pd.to_datetime('2024-01-01')
fig_bess = make_subplots(
    rows=2, cols=2, subplot_titles=zones,
    vertical_spacing=0.12, horizontal_spacing=0.08
)

summary_table = []

for i, zone in enumerate(zones):
    price_col = f'{zone}_rtm'
    df = full_df[['datetime', price_col, 'wind_mw', 'solar_mw']].copy()
    df = df[df[price_col] > 0]
    df['log_return'] = np.log(df[price_col]).diff().replace([np.inf, -np.inf], np.nan).clip(-3, 3)
    df = df.dropna(subset=['log_return'])

    df['wind_std'] = df['wind_mw'].rolling(24).std().fillna(method='bfill')
    df['solar_std'] = df['solar_mw'].rolling(24).std().fillna(method='bfill')

    df_pre = df[df['datetime'] < bess_start_date].copy()
    df_post = df[df['datetime'] >= bess_start_date].copy()

    try:
        res_pre = arch_model(df_pre['log_return'] * 100, vol='Garch', p=1, q=1).fit(disp='off')
        df_pre['vol'] = res_pre.conditional_volatility
    except:
        df_pre['vol'] = np.nan

    try:
        res_post = arch_model(df_post['log_return'] * 100, vol='Garch', p=1, q=1).fit(disp='off')
        df_post['vol'] = res_post.conditional_volatility
    except:
        df_post['vol'] = np.nan

    df_pre['year'] = df_pre['datetime'].dt.year
    df_post['year'] = df_post['datetime'].dt.year
    avg_yearly = pd.concat([df_pre, df_post]).groupby('year')['vol'].mean().reset_index()

    # Rolling average
    df_pre['rolling_vol'] = df_pre['vol'].rolling(24 * 30).mean()
    df_post['rolling_vol'] = df_post['vol'].rolling(24 * 30).mean()

    r, c = i // 2 + 1, i % 2 + 1
    color = zone_colors[zone]

    fig_bess.add_trace(go.Scatter(
        x=df_pre['datetime'], y=df_pre['rolling_vol'], mode='lines',
        name=f'{zone} Pre-BESS', line=dict(color='lightgray', width=1.2),
        showlegend=(i == 0)
    ), row=r, col=c)

    fig_bess.add_trace(go.Scatter(
        x=df_post['datetime'], y=df_post['rolling_vol'], mode='lines',
        name=f'{zone} Post-BESS', line=dict(color=color, width=2.2),
        showlegend=(i == 0)
    ), row=r, col=c)

    fig_bess.add_trace(go.Scatter(
        x=avg_yearly['year'], y=avg_yearly['vol'],
        mode='lines+text', line=dict(color='black', dash='dash'),
        text=avg_yearly['vol'].round(1), textposition='top right',
        name=f'{zone} Yearly Avg', showlegend=False
    ), row=r, col=c)

    # Summary stats and statistical tests
    mean_pre = df_pre['vol'].mean()
    mean_post = df_post['vol'].mean()
    std_pre = df_pre['vol'].std()
    std_post = df_post['vol'].std()
    _, p_ttest = ttest_ind(df_pre['vol'].dropna(), df_post['vol'].dropna(), equal_var=False)
    _, p_mw = mannwhitneyu(df_pre['vol'].dropna(), df_post['vol'].dropna(), alternative='two-sided')

    # Residual diagnostics (only on post)
    lb_pval = het_arch(res_post.resid.dropna())[1] if res_post else np.nan
    arch_pval = acorr_ljungbox(res_post.resid.dropna(), lags=[12], return_df=True)['lb_pvalue'].iloc[0] if res_post else np.nan

    summary_table.append([
        zone, round(mean_pre, 2), round(mean_post, 2),
        round(std_pre, 1), round(std_post, 1),
        round(p_ttest, 4), round(p_mw, 4), round(lb_pval, 4), round(arch_pval, 4)
    ])

# Set subplot font size and layout
for ann in fig_bess['layout']['annotations']:
    ann['font'] = dict(size=14)

fig_bess.update_layout(
    height=950,
    title="Rolling GARCH Volatility – Pre vs Post BESS",
    plot_bgcolor='white',
    font=dict(family="Arial", size=12),
    margin=dict(l=40, r=40, t=60, b=40)
)
fig_bess.show()

# Plot Wind and Solar Daily Std Dev with Different Colors
fig_ws = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=["Wind MW Daily Std Dev", "Solar MW Daily Std Dev"])
fig_ws.add_trace(go.Scatter(
    x=full_df['datetime'], y=full_df['wind_mw'].rolling(24).std(),
    line=dict(color='teal'), name="Wind Std Dev"
), row=1, col=1)
fig_ws.add_trace(go.Scatter(
    x=full_df['datetime'], y=full_df['solar_mw'].rolling(24).std(),
    line=dict(color='orange'), name="Solar Std Dev"
), row=2, col=1)
fig_ws.update_layout(title="Daily Std Dev of Wind and Solar (MW)", height=700)
fig_ws.show()

# Print statistical summary
sum_df = pd.DataFrame(summary_table, columns=[
    "Zone", "Pre-BESS Mean", "Post-BESS Mean", "Pre Std", "Post Std",
    "p-value (t-test)", "p-value (MWU)", "ARCH Test (p)", "Ljung-Box (p)"
])
sum_df.to_csv("volatility_summary_stats.csv", index=False)
print("\nStatistical Summary of Volatility Pre vs Post BESS:")
print(sum_df.to_string(index=False))


In [ ]:
# ercot_volatility_analysis.py (Post-BESS GARCH-X Only)

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from arch import arch_model

# === Load dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
full_df = pd.read_csv(file_path, parse_dates=['datetime'])

# === Define zones and colormap ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']
zone_colors = {
    'LZ_HOUSTON': 'rgba(20, 40, 90, 1.0)',
    'LZ_NORTH': 'rgba(40, 60, 120, 1.0)',
    'LZ_SOUTH': 'rgba(60, 90, 150, 1.0)',
    'LZ_WEST': 'rgba(80, 120, 180, 1.0)'
}

bess_start_date = pd.to_datetime('2024-01-01')
fig_gx = make_subplots(
    rows=2, cols=2, subplot_titles=zones,
    vertical_spacing=0.12, horizontal_spacing=0.08
)

def safe_post_garchx(data, exog_cols):
    X = data[exog_cols].copy()
    low_var_cols = [col for col in exog_cols if X[col].std() < 1e-3 or X[col].isna().all()]
    X = X.drop(columns=low_var_cols)
    if X.shape[1] == 0:
        return None, np.nan, np.nan, np.nan
    try:
        model = arch_model(data['log_return'] * 100, vol='Garch', p=1, q=1, x=X)
        result = model.fit(disp='off')
        coefs = [result.params.get(f'x{i+1}', np.nan) for i in range(len(X.columns))]
        return result, *coefs + [np.nan] * (3 - len(coefs))
    except:
        return None, np.nan, np.nan, np.nan

coeff_table = []

for i, zone in enumerate(zones):
    price_col = f'{zone}_rtm'
    df = full_df[['datetime', price_col, 'wind_mw', 'solar_mw', 'charging_mw', 'discharging_mw', 'net_output_mw']].copy()
    df = df[df[price_col] > 0]
    df['log_return'] = np.log(df[price_col]).diff().replace([np.inf, -np.inf], np.nan).clip(-3, 3)
    df = df.dropna(subset=['log_return'])

    df['wind_std'] = df['wind_mw'].rolling(24).std().fillna(method='bfill')
    df['solar_std'] = df['solar_mw'].rolling(24).std().fillna(method='bfill')
    df['bess_std'] = df['net_output_mw'].rolling(24).std().fillna(method='bfill')

    df_post = df[df['datetime'] >= bess_start_date].copy()
    res_post, wind_post, solar_post, bess_post = safe_post_garchx(df_post, ['wind_std', 'solar_std', 'bess_std'])

    if res_post:
        df_post['vol'] = res_post.conditional_volatility
    else:
        df_post['vol'] = np.nan

    coeff_table.append({
        'Zone': zone, 'Period': 'Post-BESS',
        'Wind Coef': wind_post, 'Solar Coef': solar_post, 'BESS Coef': bess_post
    })

    df_post['rolling_vol'] = df_post['vol'].rolling(24 * 30).mean()
    r, c = i // 2 + 1, i % 2 + 1
    color = zone_colors[zone]

    fig_gx.add_trace(go.Scatter(
        x=df_post['datetime'], y=df_post['rolling_vol'], mode='lines',
        name=f'{zone} Post-BESS', line=dict(color=color, width=2.5),
        showlegend=(i == 0)
    ), row=r, col=c)

fig_gx.update_layout(
    height=900,
    title="GARCH-X Volatility (Post-BESS only): Wind, Solar, BESS as Exogenous Variables",
    plot_bgcolor='white',
    font=dict(family="Arial", size=12),
    margin=dict(l=40, r=40, t=60, b=40)
)
fig_gx.show()

# === Coefficient Table ===
coef_df = pd.DataFrame(coeff_table)
print("\n=== GARCH-X Coefficients (Post-BESS Only) ===")
print(coef_df.round(4).to_string(index=False))


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor

# === Load dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
full_df = pd.read_csv(file_path, parse_dates=['datetime'])

# === Focus on one zone (e.g. LZ_HOUSTON) ===
zone = 'LZ_HOUSTON'
df = full_df[['datetime', f'{zone}_rtm', 'wind_mw', 'solar_mw', 'net_output_mw']].copy()
df = df[df[f'{zone}_rtm'] > 0]

# === Create volatility proxies (rolling std dev) ===
df['wind_std'] = df['wind_mw'].rolling(24).std().bfill()
df['solar_std'] = df['solar_mw'].rolling(24).std().bfill()
df['bess_std'] = df['net_output_mw'].rolling(24).std().bfill()

# === Filter post-BESS ===
df = df[df['datetime'] >= '2024-01-01'].copy()
df.dropna(subset=['wind_std', 'solar_std', 'bess_std'], inplace=True)

# === Correlation matrix ===
print("\n🔗 Correlation Matrix:")
corr = df[['wind_std', 'solar_std', 'bess_std']].corr()
print(corr.round(3))

# === VIF calculation ===
X = df[['wind_std', 'solar_std', 'bess_std']]
X = (X - X.mean()) / X.std()  # Standardize
vif_data = pd.DataFrame()
vif_data['Regressor'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("\n📊 Variance Inflation Factor (VIF):")
print(vif_data)

# === Optional heatmap ===
plt.figure(figsize=(5, 4))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Correlation Heatmap – Volatility Regressors")
plt.tight_layout()
plt.show()
